In [111]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense,Dropout
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss,roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [112]:
train = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\train.csv')
test = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\test.csv')
submit = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\head_submit_sample.csv')

In [113]:
#jobの欠損値処理
test.replace({'job': {'unknown':'blue-collar'}}, inplace=True)

cat_cols = ['job','marital','education','default','housing','loan','contact','month']

#one-hot-encoding
all_x = pd.concat([train,test])
all_x = pd.get_dummies(all_x, columns=cat_cols)

train = all_x.iloc[:train.shape[0], :].reset_index(drop=True)
test = all_x.iloc[train.shape[0]:, :].reset_index(drop=True)


In [114]:
un_df =  train.loc[train['poutcome'] == 'unknown']
df = train.loc[train['poutcome'] != 'unknown']

un_df.drop(['poutcome','y'], axis = 1, inplace =True)
df.drop(['y'], axis = 1, inplace =True)


df = pd.get_dummies(df, 'poutcome')

C:\Users\odoru\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [115]:
df_x = df.iloc[:,1:46]
df_y = df.loc[:,['poutcome_failure','poutcome_other','poutcome_success']]
df_test_x = un_df.drop(['id'] ,axis = 1)

In [116]:
#hold out 法
kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(df_x))[0]
tr_x,va_x = df_x.iloc[tr_idx], df_x.iloc[va_idx]
tr_y,va_y = df_y.iloc[tr_idx], df_y.iloc[va_idx] 

In [117]:
scaler = StandardScaler()
tr_x = scaler.fit_transform(tr_x)
va_x = scaler.fit_transform(va_x)
df_test_x_s = scaler.fit_transform(df_test_x)

In [118]:
model = Sequential()
model.add(Dense(256,activation='relu',input_shape=(df_x.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',
             optimizer ='adam', metrics=['accuracy'])

batch_size = 128
epochs = 100
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, restore_best_weights=True)


history  = model.fit(tr_x,tr_y,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1,validation_data=(va_x,va_y),
                    callbacks = early_stopping)

va_pred = model.predict(va_x)
score = log_loss(va_y,va_pred, eps=1e-7)
print(score)

Epoch 1/100
24/24 [==============================] - 0s 8ms/step - loss: 0.5226 - accuracy: 0.6543 - val_loss: 0.4555 - val_accuracy: 0.7043
Epoch 2/100
24/24 [==============================] - 0s 5ms/step - loss: 0.4620 - accuracy: 0.6867 - val_loss: 0.4381 - val_accuracy: 0.7063
Epoch 3/100
24/24 [==============================] - 0s 5ms/step - loss: 0.4448 - accuracy: 0.6947 - val_loss: 0.4322 - val_accuracy: 0.7073
Epoch 4/100
24/24 [==============================] - 0s 5ms/step - loss: 0.4399 - accuracy: 0.6973 - val_loss: 0.4305 - val_accuracy: 0.7163
Epoch 5/100
24/24 [==============================] - 0s 5ms/step - loss: 0.4360 - accuracy: 0.7040 - val_loss: 0.4303 - val_accuracy: 0.7203
Epoch 6/100
24/24 [==============================] - 0s 5ms/step - loss: 0.4287 - accuracy: 0.7070 - val_loss: 0.4304 - val_accuracy: 0.7163
Epoch 7/100
24/24 [==============================] - 0s 5ms/step - loss: 0.4237 - accuracy: 0.7050 - val_loss: 0.4288 - val_accuracy: 0.7183
Epoch 8/100
2

In [119]:
test_pred = model.predict(df_test_x_s)

In [120]:
a = test_pred.argmax(1)

In [121]:
# トレーニングデータの完成
df_test_x['poutcome'] = a

df_concat_train = pd.concat([df_x, df_y], axis = 1)
df_concat_train = pd.concat([df_concat_train, df_test_x], axis = 0)
df_concat_train = df_concat_train.sort_index()

In [122]:
df_test_x

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome
1,29,43027,22,158,2,702,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,48,42005,3,177,1,273,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,30,-6160,18,158,1,632,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,33,88282,27,345,1,272,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
7,30,59830,27,184,1,639,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27095,37,26661,27,345,4,425,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
27096,35,42150,27,121,1,719,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
27097,35,34531,28,177,2,121,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
27098,30,99621,27,121,1,100,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [123]:
a = pd.get_dummies(df_concat_train['poutcome'])
a.columns = ['failure','other','success']
a

,failure,other,success
0,0,0,0
1,1,0,0
2,0,0,0
3,0,0,0
4,1,0,0
...,...,...,...
27095,1,0,0
27096,1,0,0
27097,1,0,0
27098,1,0,0


In [125]:
a.to_csv('NN_poutcome.csv',index=False)